In [14]:
# Cell 1: Imports and configuration
import pickle
import json
import requests
from typing import List, Dict
import random
from tqdm import tqdm
import re
from transformers import AutoTokenizer

# vLLM API configuration
VLLM_URL = "http://0.0.0.0:8000/v1/chat/completions"
HEADERS = {"Content-Type": "application/json"}

GENERATION_CONFIG = {
    "model": "meta-llama/llama-3.3-70b-instruct",
    "temperature": 0.8,
    "max_tokens": 4096,  # Increased for multi-turn conversations
    "top_p": 0.95
}

# Load processed conversations
#with open('data/processed_conversations.pkl', 'rb') as f:
#    conversations = pickle.load(f)

In [15]:
from datasets import load_dataset

# Load private dataset (requires authentication)
dataset = load_dataset("mikeriess/LM_anonymized")
df = dataset['train'].to_pandas()
conversations = dict(zip(df['conversation_id'], df['text']))
conversations


Using the latest cached version of the dataset since mikeriess/LM_anonymized couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at C:\Users\miker\.cache\huggingface\datasets\mikeriess___lm_anonymized\default\0.0.0\76670ebc4eb4b637c669be403b31dbe7e60a306e (last modified on Mon Feb 17 16:19:04 2025).


{176: '1: Hey. Er der en måde man ved et enkelt klik kan bypasse alle plug-ins (effekter og lign, ikke AU\'s) på én gang? Hvis man nu fx har optaget noget band som der senere skal ligges vokal på og man har tid til at mixe lidt inden vokal-session\'en er det rimeligt irriterende at skulle bypass\'e hvert enkelt plug-in for at undgå forsinkelse på vokalen når man indspiller den... Er der en nemmere måde? [PER]\n2: Du kan f.eks. sætte en anden kanal (i Audio environmentet) op til det du gerne vil med de samme input/output, og når du så har brug for at optage så vælger du bare denne kanal i Arrange vinduet i stedet for den der er sat op til mix. See? Altså du klikker bare på sporet ude til venstre i Arrange og vælger den anden kanal...\n1: I see... tror jeg... men det skal jeg jo så gøre for hvert enkelt spor jeg har puttet plug-ins på - hvilket jo nemt bliver en 8-10 stykker med et fuldvoksent band... noget nemmere?\n6: Du kan bounce det hele til to spor og bruge en ny session til vocali

In [2]:
# Cell 2: Define generation functions
def generate_synthetic_conversation(context: str, question_id: str) -> List[Dict[str, str]]:
    """Generate a multi-turn synthetic conversation using the context"""

    system_prompt = """Du er en dansk samtalegenerator. Generer en realistisk dialog med flere udvekslinger 
    mellem 2 personer på dansk, inspireret af følgende samtale som kontekst.
    
    VIGTIGT: Den første besked SKAL være et spørgsmål, og svarene skal være i en venlig tone og aldrig indeholde URLs. Hvis der er et spørgsmål om at finde en URL, så skal du i stedet svare med relevante søgeord.
    
    Dialogen skal have mindst 1 udveksling frem og tilbage, men gerne flere hvis konteksten giver anledning til det.
    Format hver besked som:
    Person 1: [spørgsmål]
    Person 2: [svar]
    Person 1: [opfølgende spørgsmål eller udsagn]
    Person 2: [svar]
    ...
    
    Kontekst:
    {context}
    
    Generer en ny, anderledes dialog omhandlende samme problemstilling:"""
    
    try:
        response = requests.post(
            VLLM_URL,
            headers=HEADERS,
            json={
                "messages": [{"role": "user", "content": system_prompt.format(context=context)}],
                **GENERATION_CONFIG
            },
            timeout=10
        )
        response.raise_for_status()
        
        dialogue = response.json()['choices'][0]['message']['content']
        return parse_multi_turn_dialogue(dialogue, question_id)
        
    except Exception as e:
        print(f"Error generating conversation: {e}")
        return None

def parse_multi_turn_dialogue(dialogue: str, question_id: str) -> List[Dict[str, str]]:
    """Parse the generated dialogue into a list of message dictionaries"""
    messages = []
    lines = [line.strip() for line in dialogue.split('\n') if line.strip()]
    
    current_turn = 1
    # Ensure first message is a question
    for i, line in enumerate(lines):
        if ':' in line:
            speaker, content = line.split(':', 1)
            content = content.strip()
            
            # For first message, verify it's a question
            if i == 0 and '?' not in content:
                # Skip this dialogue as it doesn't start with a question
                return None
            
            role = "user" if "Person 1" in speaker else "assistant"
            messages.append({
                "role": role,
                "content": content,
                "question_id": question_id,
                "turn": current_turn
            })
            
            # Increment turn after each assistant response
            if role == "assistant":
                current_turn += 1
    
    return messages if messages and '?' in messages[0]['content'] else None

def format_as_instruction_data(messages: List[Dict[str, str]]) -> List[Dict[str, List]]:
    """Format messages as instruction data for TRL fine-tuning"""
    instruction_examples = []
    
    # Create examples from consecutive turns
    for i in range(0, len(messages)-1, 2):
        if i+1 < len(messages):
            example = {
                "question_id": messages[i]['question_id'],
                "turn": messages[i]['turn'],  # Use the turn number from the user message
                "prompt": [messages[i]],  # User message
                "completion": [messages[i+1]]  # Assistant response
            }
            instruction_examples.append(example)
    
    return instruction_examples

In [3]:
# Cell 3: Generate synthetic dataset
import uuid

instruction_dataset = []
max_conversations = 100  # Adjust as needed
min_conv_length = 50

print("Generating synthetic conversations...")
for conv_id, conv_text in tqdm(list(conversations.items())[:max_conversations]):
    if len(conv_text) < min_conv_length:
        continue
        
    # Generate unique ID for this conversation
    question_id = str(uuid.uuid4().hex)
    
    # Generate synthetic multi-turn dialogue
    messages = generate_synthetic_conversation(conv_text, question_id)
    if messages and len(messages) >= 4:  # Ensure we have at least 2 turns
        # Format as instruction examples
        examples = format_as_instruction_data(messages)
        instruction_dataset.extend(examples)

print(f"Generated {len(instruction_dataset)} instruction examples")

# Save the dataset
if instruction_dataset:
    with open('danish_instruction_dataset.pkl', 'wb') as f:
        pickle.dump(instruction_dataset, f)

    # Print a few examples
    print("\nExample instruction data:")
    for example in instruction_dataset[:2]:
        print(f"\nQuestion ID: {example['question_id']}")
        print("\nPrompt messages:")
        for msg in example['prompt']:
            print(f"Role: {msg['role']}, Content: {msg['content']}")
        print("\nCompletion messages:")
        for msg in example['completion']:
            print(f"Role: {msg['role']}, Content: {msg['content']}")

Generating synthetic conversations...


 28%|██▊       | 28/100 [01:34<04:59,  4.16s/it]

Error generating conversation: 400 Client Error: Bad Request for url: http://itx:8000/v1/chat/completions


 48%|████▊     | 48/100 [02:41<03:33,  4.11s/it]

Error generating conversation: 400 Client Error: Bad Request for url: http://itx:8000/v1/chat/completions


...

Error generating conversation: 400 Client Error: Bad Request for url: http://itx:8000/v1/chat/completions


 95%|█████████▌| 95/100 [05:06<00:10,  2.02s/it]

Error generating conversation: 400 Client Error: Bad Request for url: http://itx:8000/v1/chat/completions


100%|██████████| 100/100 [05:20<00:00,  3.20s/it]

Error generating conversation: 400 Client Error: Bad Request for url: http://itx:8000/v1/chat/completions
Generated 317 instruction examples

Example instruction data:

Prompt messages:
Role: user, Content: Hej, har en lille problem med plug-in'er. Jeg prøver at optage vokal, men det ender altid med forsinkelse på den vokal-monitor.

Completion messages:
Role: assistant, Content: Hmm, det kan være frustrerende! Er det især på den side af plug-in'erne, eller når du bruger dem i general?

Prompt messages:
Role: user, Content: Ja, det er især med plug-in'er i den øvre del af mixet. Jeg har en masse forskellige plugins på forskellige spor. Det er ikke en lang session, men når jeg prøver at optage vokal, er der en forsinkelse.

Completion messages:
Role: assistant, Content: Hmm, det er måske fordi du ikke mute'r plug-in'erne på de andre spor?


In [5]:
instruction_dataset

[{'prompt': [{'role': 'user',
    'content': "Hej, har en lille problem med plug-in'er. Jeg prøver at optage vokal, men det ender altid med forsinkelse på den vokal-monitor."}],
  'completion': [{'role': 'assistant',
    'content': "Hmm, det kan være frustrerende! Er det især på den side af plug-in'erne, eller når du bruger dem i general?"}]},
 {'prompt': [{'role': 'user',
    'content': "Ja, det er især med plug-in'er i den øvre del af mixet. Jeg har en masse forskellige plugins på forskellige spor. Det er ikke en lang session, men når jeg prøver at optage vokal, er der en forsinkelse."}],
  'completion': [{'role': 'assistant',
    'content': "Hmm, det er måske fordi du ikke mute'r plug-in'erne på de andre spor?"}]},
 {'prompt': [{'role': 'user',
    'content': "Hmm, det var en tanke, men jeg har ikke umiddelbart haft succes med det. Jeg har prøvet at bounce alle plug-in'er, men det giver ikke den ønskede effekt."}],
  'completion': [{'role': 'assistant',
    'content': 'Er du sikker 

In [ ]:
# Cell 4: Format and upload dataset to HuggingFace Hub
from datasets import Dataset
import pandas as pd

def prepare_dataset_for_upload(instruction_data):
    """Convert our instruction format to match chatbot arena structure"""
    formatted_data = []
    
    for example in instruction_data:
        # Get existing IDs and turn number
        question_id = example['question_id']
        turn = example['turn']
        
        # Format conversations in the arena style
        conversation = [
            {"role": "user", "content": example['prompt'][0]['content']},
            {"role": "assistant", "content": example['completion'][0]['content']}
        ]
        
        formatted_data.append({
            'question_id': question_id,
            'model_generator': GENERATION_CONFIG["model"],  # Our generation model
            'conversation': conversation,
            'language': 'Danish',
            'turn': turn,  # Using the turn number from our data
        })
    
    return formatted_data

# Prepare the dataset
formatted_data = prepare_dataset_for_upload(instruction_dataset)

# Convert to HF Dataset
hf_dataset = Dataset.from_pandas(pd.DataFrame(formatted_data))

# Push to the Hub as a private dataset
dataset_name = "mikeriess/LM_DA_test1"
hf_dataset.push_to_hub(
    f"mikeriess/{dataset_name}", 
    private=True,
    token=True
)

print(f"Dataset uploaded to: https://huggingface.co/datasets/mikeriess/{dataset_name}")

# Print a sample to verify format
print("\nSample entry:")
sample = formatted_data[0]
print(f"Question ID: {sample['question_id']}")
print(f"Turn: {sample['turn']}")
print("Conversation:")
for msg in sample['conversation_a']:
    print(f"{msg['role']}: {msg['content']}")